In [1]:
import sys
sys.path.append('../')

import tquant as tq
from engine.globalconfig import GlobalConfig
from engine.loader.curveload import MarketDataLoader

from datetime import date
import tensorflow as tf
import pandas as pd

# Settings

In [2]:
config = GlobalConfig.from_json()
mdl = MarketDataLoader(config)

In [3]:
tq.Settings.evaluation_date

datetime.date(2023, 9, 29)

In [4]:
calendar = tq.TARGET()
daycounter = tq.DayCounter(tq.DayCounterConvention.Actual360)
evaluation_date = tq.Settings.evaluation_date

# Market

In [5]:
rates = mdl.ir_eur_curve_estr['quote'].values
times = mdl.ir_eur_curve_estr['daycount'].values/365

rates_6m = mdl.ir_eur_curve_6m['quote'].values
times_6m = mdl.ir_eur_curve_6m['daycount'].values/365

In [6]:
estr_curve = tq.RateCurve(times, rates)
eur6m_curve = tq.RateCurve(times_6m, rates_6m)

# Floating coupon

In [7]:
d1 = calendar.advance(evaluation_date,
                         6,
                         tq.TimeUnit.Months,
                         tq.BusinessDayConvention.ModifiedFollowing)

index = tq.IborIndex('EUR6M',
             calendar,
             6,
             tq.TimeUnit.Months)
gearing = 1
spread = 0
nominal = 1e6

floating_coupon = tq.FloatingCoupon(d1,
                                    nominal,
                                    evaluation_date,
                                    d1,
                                    index,
                                    gearing,
                                    spread,
                                    evaluation_date,
                                    d1,
                                    daycounter
                                    )

In [8]:
float_pricer = tq.FloatingCouponDiscounting(floating_coupon, False)

In [9]:
npv, tape = float_pricer.price_aad(estr_curve, eur6m_curve, tq.Settings.evaluation_date)
sensitivities = tape.gradient(npv, [estr_curve.rates, eur6m_curve.rates])

In [10]:
sensy = pd.DataFrame([estr_curve.pillars.tolist(), sensitivities[0], sensitivities[1]]).T
sensy.columns = ['time', 'sensy_estr', 'sensy_eur6m']
sensy[sensy['time'] > 0.2].head(10)

,time,sensy_estr,sensy_eur6m
8,0.249315,NaN,NaN
9,0.334247,None,"tf.Tensor(483101.04394118447, shape=(), dtype=..."
10,0.419178,None,"tf.Tensor(20078.367153468203, shape=(), dtype=..."
11,0.49589,"tf.Tensor(-9497.84399362655, shape=(), dtype=f...",None
12,0.583562,"tf.Tensor(-809.7549518824293, shape=(), dtype=...",None
13,0.665753,NaN,NaN
14,0.747945,NaN,NaN
15,0.832877,NaN,NaN
16,0.917808,NaN,NaN
17,1.005479,NaN,NaN


In [11]:
npv

<tf.Tensor: shape=(), dtype=float64, numpy=20501.301770073107>